<a href="https://colab.research.google.com/github/biherrerap/GaussianBlur-CUDA/blob/master/CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!echo CUDA AND NVIDIA INSTALLATION
!echo NOTE: THERE IS A QUESTION IN THE INSTALLATION, PLEASE DO NOT FORGET TO ANSWER IT (YOU CAN CHOOSE Y)
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb;
!dpkg -i cuda-repo-ubuntu1604_8.0.61-1_amd64.deb;
!apt-get update -qq;
!apt-get install cuda-8.0;
!ln -sf /usr/local/cuda-8.0 /usr/local/cuda

CUDA AND NVIDIA INSTALLATION
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `echo NOTE: THERE IS A QUESTION IN THE INSTALLATION, PLEASE DO NOT FORGET TO ANSWER IT (YOU CAN CHOOSE Y)'
--2019-06-20 13:31:11--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_8.0.61-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 192.229.232.112, 2606:2800:247:2063:46e:21d:825:102e
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|192.229.232.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2690 (2.6K) [application/x-deb]
Saving to: ‘cuda-repo-ubuntu1604_8.0.61-1_amd64.deb’

cuda-repo-ubuntu160 100%[===================>]   2.63K  --.-KB/s    in 0s      

2019-06-20 13:31:11 (167 MB/s) - ‘cuda-repo-ubuntu1604_8.0.61-1_amd64.deb’ saved [2690/2690]

(Reading database ... 140915 files and directories currently installed.)
Preparing t

In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda/lib'

!apt-get install gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!clear

ln: failed to create symbolic link '/usr/local/cuda/bin/gcc': File exists
ln: failed to create symbolic link '/usr/local/cuda/bin/g++': File exists
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-sorw3lc3
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-sorw3lc3
  Stored in directory: /tmp/pip-ephem-wheel-cache-xg5c2q5g/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [0]:
!echo NVIDIA CUDA AND DRIVES VERIFICATION
%cd /usr/local/cuda/samples/1_Utilities/deviceQuery/
!ls
!make
!./deviceQuery
!nvcc --version

NVIDIA CUDA AND DRIVES VERIFICATION
/usr/local/cuda-8.0/samples/1_Utilities/deviceQuery
cuda-repo-ubuntu1604_8.0.61-1_amd64.deb  deviceQuery.o	Makefile
deviceQuery				 main		NsightEclipse.xml
deviceQuery.cpp				 main.cu	readme.txt
make: Nothing to be done for 'all'.
./deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla K80"
  CUDA Driver Version / Runtime Version          10.0 / 8.0
  CUDA Capability Major/Minor version number:    3.7
  Total amount of global memory:                 11441 MBytes (11996954624 bytes)
  (13) Multiprocessors, (192) CUDA Cores/MP:     2496 CUDA Cores
  GPU Max Clock rate:                            824 MHz (0.82 GHz)
  Memory Clock rate:                             2505 Mhz
  Memory Bus Width:                              384-bit
  L2 Cache Size:                                 1572864 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(65536), 2D=(65536, 65536

In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-gmav_fg1
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-gmav_fg1
  Stored in directory: /tmp/pip-ephem-wheel-cache-3iswke7k/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [0]:
%reload_ext nvcc_plugin

created output directory at /usr/local/cuda-8.0/samples/1_Utilities/deviceQuery/src
Out bin /usr/local/cuda-8.0/samples/1_Utilities/deviceQuery/result.out


#La foto se tiene que subir a: 
/usr/local/cuda-8.0/samples/1_Utilities/deviceQuery

Y el nombre de la foto se especifica en el programa, por ahora estoy usando la de 720p.png

In [0]:
%%writefile main.cu

#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <stdarg.h>
#include <vector>
#include <cmath>
#include <time.h>
#include <helper_cuda.h>
#define PNG_DEBUG 3
#include <png.h>

using std::vector;

typedef vector<double> Array;
typedef vector<Array> Matrix;
typedef vector<Matrix> Image;

Matrix filter;
Image image;
Image newImage;
int num_threads;

void abort_(const char * s, ...)
{
        va_list args;
        va_start(args, s);
        vfprintf(stderr, s, args);
        fprintf(stderr, "\n");
        va_end(args);
        abort();
}

int GetDeviceCount(){
    int deviceCount = 0;
    cudaError_t error_id = cudaGetDeviceCount(&deviceCount);
    return deviceCount;
}

int x, y;

int width, height;
png_byte color_type;
png_byte bit_depth;

png_structp png_ptr;
png_infop info_ptr;
int number_of_passes;
png_bytep * row_pointers;

void read_png_file(char* file_name)
{
        char header[8];    // 8 is the maximum size that can be checked

        /* open file and test for it being a png */
        FILE *fp = fopen(file_name, "rb");
        if (!fp)
                abort_("[read_png_file] File %s could not be opened for reading", file_name);
        fread(header, 1, 8, fp);
        //if (png_sig_cmp(header, 0, 8))
        //        abort_("[read_png_file] File %s is not recognized as a PNG file", file_name);


        /* initialize stuff */
        png_ptr = png_create_read_struct(PNG_LIBPNG_VER_STRING, NULL, NULL, NULL);

        if (!png_ptr)
                abort_("[read_png_file] png_create_read_struct failed");

        info_ptr = png_create_info_struct(png_ptr);
        if (!info_ptr)
                abort_("[read_png_file] png_create_info_struct failed");

        if (setjmp(png_jmpbuf(png_ptr)))
                abort_("[read_png_file] Error during init_io");

        png_init_io(png_ptr, fp);
        png_set_sig_bytes(png_ptr, 8);

        png_read_info(png_ptr, info_ptr);

        width = png_get_image_width(png_ptr, info_ptr);
        height = png_get_image_height(png_ptr, info_ptr);
        color_type = png_get_color_type(png_ptr, info_ptr);
        bit_depth = png_get_bit_depth(png_ptr, info_ptr);

        number_of_passes = png_set_interlace_handling(png_ptr);
        png_read_update_info(png_ptr, info_ptr);


        /* read file */
        if (setjmp(png_jmpbuf(png_ptr)))
                abort_("[read_png_file] Error during read_image");

        row_pointers = (png_bytep*) malloc(sizeof(png_bytep) * height);
        for (y=0; y<height; y++)
                row_pointers[y] = (png_byte*) malloc(png_get_rowbytes(png_ptr,info_ptr));

        png_read_image(png_ptr, row_pointers);

        fclose(fp);
}


void write_png_file(char* file_name)
{
        /* create file */
        FILE *fp = fopen(file_name, "wb");
        if (!fp)
                abort_("[write_png_file] File %s could not be opened for writing", file_name);


        /* initialize stuff */
        png_ptr = png_create_write_struct(PNG_LIBPNG_VER_STRING, NULL, NULL, NULL);

        if (!png_ptr)
                abort_("[write_png_file] png_create_write_struct failed");

        info_ptr = png_create_info_struct(png_ptr);
        if (!info_ptr)
                abort_("[write_png_file] png_create_info_struct failed");

        if (setjmp(png_jmpbuf(png_ptr)))
                abort_("[write_png_file] Error during init_io");

        png_init_io(png_ptr, fp);


        /* write header */
        if (setjmp(png_jmpbuf(png_ptr)))
                abort_("[write_png_file] Error during writing header");

        png_set_IHDR(png_ptr, info_ptr, width, height,
                     bit_depth, color_type, PNG_INTERLACE_NONE,
                     PNG_COMPRESSION_TYPE_BASE, PNG_FILTER_TYPE_BASE);

        png_write_info(png_ptr, info_ptr);


        /* write bytes */
        if (setjmp(png_jmpbuf(png_ptr)))
                abort_("[write_png_file] Error during writing bytes");

        png_write_image(png_ptr, row_pointers);


        /* end write */
        if (setjmp(png_jmpbuf(png_ptr)))
                abort_("[write_png_file] Error during end of write");

        png_write_end(png_ptr, NULL);

        /* cleanup heap allocation */
        for (y=0; y<height; y++)
                free(row_pointers[y]);
        free(row_pointers);

        fclose(fp);
}

Matrix getGaussian(int height, int width, double sigma)
{
    Matrix kernel(height, Array(width));
    double sum=0.0;
    int i,j;

    for (i=0 ; i<height ; i++) {
        for (j=0 ; j<width ; j++) {
            kernel[i][j] = exp(-(i*i+j*j)/(2*sigma*sigma))/(2*M_PI*sigma*sigma);
            sum += kernel[i][j];
        }
    }

    for (i=0 ; i<height ; i++) {
        for (j=0 ; j<width ; j++) {
            kernel[i][j] /= sum;
        }
    }

    return kernel;
}

__global__ void vectorAdd(double *image_d1, double *kernel, double *output, int width, int height, int kernelSize, int totalThreads)
{
    int id = blockDim.x * blockIdx.x + threadIdx.x;
    //printf("blockDim is: %d blockId is: %d threadId is: %d id is: %d\n", blockDim.x, blockIdx.x, threadIdx.x, id);
    //printf("width: %d height: %d ", width, height);

    int filterHeight = kernelSize;
    int filterWidth = kernelSize;
    int newImageHeight = height-kernelSize+1;
    //int newImageHeight = height;
    int newImageWidth = width-kernelSize+1;
    int d,i,j,h,w;
    
    //int fromY = (newImageHeight / totalThreads)*id;
    //int toY = id != totalThreads-1 ? fromY + (newImageHeight / totalThreads) : newImageHeight;
    
    //int fromX = (newImageWidth / totalThreads)*id;
    //int toX = id != totalThreads-1 ? fromX + (newImageWidth / totalThreads) : newImageWidth;
    
    int fromY = ((newImageHeight + totalThreads - 1) / totalThreads)*id;
    int toY = id != totalThreads-1 ? fromY + ((newImageHeight + totalThreads - 1) / totalThreads) : newImageHeight;

    int fromX = ((newImageWidth + totalThreads - 1) / totalThreads)*id;
    int toX = id != totalThreads-1 ? fromX + ((newImageWidth + totalThreads - 1) / totalThreads) : newImageWidth;
    
    //printf("ID: %d #Blocks %d From %d To %d\n",id,(toX-fromX), fromX, toX);
    
    if(toX > newImageWidth) return;
    
    int numElements = sizeof(double) * 4 * newImageHeight * newImageWidth;
        
    for (i=0; i<newImageHeight ; i++) {
        for (j=fromX ; j<toX ; j++) {
            for(int k=0; k<4; k++){
                output[i*newImageWidth*4 + j*4 + k] = 0;
                for (h=i ; h<i+filterHeight ; h++) {
                    for (w=j ; w<j+filterWidth ; w++) {
                        output[i*newImageWidth*4 + j*4 + k] += kernel[(h-i) * kernelSize + (w-j)]*image_d1[h*width*4 + w*4 + k];
                    }
                }
            }
        }
    }
    
    printf("");
}


void process_file(int kernel,int blocks,int threads)
{
    if (png_get_color_type(png_ptr, info_ptr) == PNG_COLOR_TYPE_RGB)
      abort_("[process_file] input file is PNG_COLOR_TYPE_RGB but must be PNG_COLOR_TYPE_RGBA "
        "(lacks the alpha channel)");

    if (png_get_color_type(png_ptr, info_ptr) != PNG_COLOR_TYPE_RGBA)
      abort_("[process_file] color_type of input file must be PNG_COLOR_TYPE_RGBA (%d) (is %d)",
        PNG_COLOR_TYPE_RGBA, png_get_color_type(png_ptr, info_ptr));
    
    int device;
    cudaGetDeviceCount(&device);
    cudaSetDevice(device);
    cudaDeviceProp deviceProp;
    cudaGetDeviceProperties(&deviceProp, device);
    
    int err;
    
    int multiprocessorNum =  deviceProp.multiProcessorCount;
    int cores_mp = _ConvertSMVer2Cores(deviceProp.major, deviceProp.minor);
    int cores = _ConvertSMVer2Cores(deviceProp.major, deviceProp.minor) *deviceProp.multiProcessorCount;
    int newWidth = (width-kernel + 1);
    int newHeight = (height-kernel + 1);
    
    int size = sizeof(double) * 4 * width * height;
    int kernelSize = sizeof(double) * kernel * kernel;
    int outputSize = sizeof(double) * 4 * newHeight * newWidth;
    
    double *image_h1 = (double *)malloc(size);
    double *image_d1 = NULL;
    err = cudaMalloc((void **)&image_d1, size);
    double *kernel_h1 = (double *)malloc(kernelSize);
    double *kernel_d1 = NULL;
    err = cudaMalloc(&kernel_d1, kernelSize);

    double *output_h = (double *)malloc(outputSize);
    double *output_d = NULL;
    err = cudaMalloc((void **)&output_d, outputSize);
   
    // Initialize the HOST image
    for (y=0; y<height; y++) {
        png_byte* row = row_pointers[y];
        for (x=0; x<width; x++) {   
            png_byte* ptr = &(row[x*4]);
            for(int i=0; i<4; i++){
                image_h1[y*width*4 + x*4 + i] = (double)ptr[i];
            }
        }
    }
    
    // Copy the HOST image to DEVICE image
    err = cudaMemcpy(image_d1, image_h1, size, cudaMemcpyHostToDevice);
    
    Matrix filter = getGaussian(kernel, kernel, 10.0);
    kernel_h1 = new double[kernel];
    for(int i=0; i<kernel; i++){
        for(int j=0; j<kernel; j++){ 
            kernel_h1[i*kernel + j] = filter[i][j];
        }
    }

    err = cudaMemcpy(kernel_d1, kernel_h1, kernelSize, cudaMemcpyHostToDevice);
    
    // vectorAdd<<<multiprocessorNum, cores_mp>>>(image_d1, kernel_d1, output_d, width, height, kernel);
    vectorAdd<<<blocks, threads>>>(image_d1, kernel_d1, output_d, width, height, kernel, blocks*threads);
    
    err = cudaMemcpy(image_h1, image_d1, size, cudaMemcpyDeviceToHost);
    err = cudaMemcpy(output_h, output_d, outputSize, cudaMemcpyDeviceToHost);
    
    for (y=0; y<newHeight; y++) {
        png_byte* row = row_pointers[y];
        for (x=0; x<newWidth; x++) {   
            png_byte* ptr = &(row[x*4]);
            for(int i=0; i<4; i++){
                 ptr[i] = output_h[y*newWidth*4 + x*4 + i];
            }
        }
    }
}

int main(int argc, char **argv)
{
    if (argc != 6)
      abort_("Usage: program_name <file_in> <file_out> <kernel size> <blocks number> <threads number>");
    
    int kernel = atoi(argv[3]);
    int blocks = atoi(argv[4]);
    int threads = atoi(argv[5]);
    
  	read_png_file(argv[1]);
    process_file(kernel, blocks, threads);
    write_png_file(argv[2]);

}

Overwriting main.cu


In [0]:
!nvcc main.cu -o main -I /usr/local/cuda/samples/common/inc -lpng

nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
main.cu(195): warning: variable "d" was declared but never referenced

main.cu(204): warning: variable "toY" was declared but never referenced

main.cu(213): warning: variable "numElements" was declared but never referenced

main.cu(248): warning: variable "err" was set but never used

main.cu(250): warning: variable "multiprocessorNum" was declared but never referenced

main.cu(195): warning: variable "d" was declared but never referenced

main.cu(204): warning: variable "toY" was declared but never referenced

main.cu(213): warning: variable "numElements" was declared but never referenced

main.cu(248): warning: variable "err" was set but never used

main.cu(250): warning: variable "multiprocessorNum" was declared but never referenced



In [0]:
!time ./main 4k.png output.png 7 1 256



MapSMtoCores for SM 1.0 is undefined.  Default to use 128 Cores/SM
MapSMtoCores for SM 1.0 is undefined.  Default to use 128 Cores/SM

real	0m12.500s
user	0m7.565s
sys	0m4.918s


In [0]:
!time ./main 1080p.png output.png 7 10 1
!time ./main 1080p.png output.png 7 10 64
!time ./main 1080p.png output.png 7 10 128
!time ./main 1080p.png output.png 7 10 256

ID: 8 #Blocks 192 From 1536 To 1728
ID: 9 #Blocks 186 From 1728 To 1914
ID: 4 #Blocks 192 From 768 To 960
ID: 3 #Blocks 192 From 576 To 768
ID: 0 #Blocks 192 From 0 To 192
ID: 1 #Blocks 192 From 192 To 384
ID: 5 #Blocks 192 From 960 To 1152
ID: 7 #Blocks 192 From 1344 To 1536
ID: 6 #Blocks 192 From 1152 To 1344
ID: 2 #Blocks 192 From 384 To 576

real	0m15.516s
user	0m8.292s
sys	0m7.199s
ID: 576 #Blocks 3 From 1728 To 1731
ID: 577 #Blocks 3 From 1731 To 1734
ID: 578 #Blocks 3 From 1734 To 1737
ID: 579 #Blocks 3 From 1737 To 1740
ID: 580 #Blocks 3 From 1740 To 1743
ID: 581 #Blocks 3 From 1743 To 1746
ID: 582 #Blocks 3 From 1746 To 1749
ID: 583 #Blocks 3 From 1749 To 1752
ID: 584 #Blocks 3 From 1752 To 1755
ID: 585 #Blocks 3 From 1755 To 1758
ID: 586 #Blocks 3 From 1758 To 1761
ID: 587 #Blocks 3 From 1761 To 1764
ID: 588 #Blocks 3 From 1764 To 1767
ID: 589 #Blocks 3 From 1767 To 1770
ID: 590 #Blocks 3 From 1770 To 1773
ID: 591 #Blocks 3 From 1773 To 1776
ID: 592 #Blocks 3 From 1776 To 177

In [0]:
!time ./main 1080p.png output.png 5 40 1

!time ./main 1080p.png output.png 5 40 64

!time ./main 1080p.png output.png 5 40 128

!time ./main 1080p.png output.png 5 40 256


real	0m3.205s
user	0m2.131s
sys	0m1.057s

real	0m1.409s
user	0m1.188s
sys	0m0.215s

real	0m1.713s
user	0m0.425s
sys	0m1.220s

real	0m1.088s
user	0m0.440s
sys	0m0.629s


In [0]:
!time ./main 4k.png output.png 3 40 1

!time ./main 1080p.png output.png 3 40 64

!time ./main 1080p.png output.png 3 40 128

!time ./main 1080p.png output.png 3 40 256


real	0m2.148s
user	0m1.577s
sys	0m0.566s

real	0m1.387s
user	0m1.195s
sys	0m0.182s

real	0m1.599s
user	0m0.406s
sys	0m1.179s

real	0m1.042s
user	0m0.409s
sys	0m0.519s
